In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from cf_model import MLPContentFilter
from dataset import FuseDataset, Encoder, split_data
import import_ipynb
import db
from torch.utils.data import DataLoader
from tqdm import tqdm
import random

importing Jupyter notebook from db.ipynb
add				fuse_encoder.pkl	README.md
cf_model.py			holocenemodels		requirements.txt
col_encoder.pkl			indie_letterboxd.db	send_data.py
con_encoder.pkl			indie_letterboxd_v2.db	server
config.py			letterboxd		test.db
dataset.py			letterboxd.db		testsources.ipynb
db.ipynb			main.ipynb		todo.txt
Dockerfile			main.py			train_colfilter.ipynb
done.txt			matching.ipynb		train_confilter.ipynb
encoder.pkl			menv			user.py
fdstests.ipynb			mlruns			wandb
filemanager.py			models			webapp
final_col_model_checkpoint.pth	movie.py		web_spider.py
final_con_model_checkpoint.pth	__pycache__
fused_filter_train.ipynb	q.py


In [2]:
DB_NAME = 'indie_letterboxd_v2'
db.display_size(DB_NAME)

Users: 2400

Movies: 771

Reviews: 18504


# Import from sqlite3

In [3]:
import sqlite3

# Connect to your database
conn = sqlite3.connect(DB_NAME+'.db')
cursor = conn.cursor()

# SQL query
query = '''
            SELECT 
                Users.user_id,
                Users.name AS user_name,
                GROUP_CONCAT(DISTINCT Genres.genre_id) AS genre_ids,
                GROUP_CONCAT(DISTINCT Genres.name) AS genre_names,
                Countries.country_id,
                Countries.country_name,
                Movies.movie_id,
                Movies.title AS movie_name,
                Reviews.rating
            FROM 
                Reviews
            JOIN
                Users ON Reviews.user_id = Users.user_id
            JOIN 
                Movies ON Reviews.movie_id = Movies.movie_id
            JOIN 
                Countries ON Movies.country_id = Countries.country_id
            LEFT JOIN 
                MoviesGenres ON Movies.movie_id = MoviesGenres.movie_id
            LEFT JOIN 
                Genres ON MoviesGenres.genre_id = Genres.genre_id
            GROUP BY 
                Reviews.review_id, Reviews.rating
            ORDER BY 
                Movies.movie_id, Users.user_id

        '''

# Execute the query
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Close the connection
conn.close()

In [4]:
len(results)

18504

In [5]:
results[0]

('0381a2ce-de6e-4a92-8117-8a290833c219',
 'Terry Roberson',
 '2,9',
 'Drama,Romance',
 29,
 'South Korea',
 '001e5226-4f9f-4e01-a824-016ddcd62aad',
 'Endings Beginnings',
 0.4)

In [6]:
list_of_lists = [list(t) for t in results]

# Shuffle the list of lists
random.shuffle(list_of_lists)

# Convert the shuffled list of lists back to a list of tuples
data = [tuple(lst) for lst in list_of_lists]

len(data)

18504

In [7]:
#test data is holdout for Kfolds cross data validation evaluation.
train_data, test_data = split_data(data)
train_data, validation_data = split_data(train_data,0.1)

In [8]:
#user,genre,country
users_data = [did[1] for did in train_data]
genres_data = [did[3].split(',') for did in train_data]#<---- need list type

countries_data = [did[5] for did in train_data]
movies_data = [did[7] for did in train_data]

ratings_data = [did[-1] for did in train_data]
ratings_data[:10]

[0.9, 0.8, 0.4, 0.8, 0.5, 1, 0.6, 1, 1, 0.7]

In [9]:
# only from the training data.
encoder = Encoder(users=users_data,movies=movies_data,genres=genres_data,countries=countries_data)
import pickle
# Assuming 'encoder' is your encoder object
with open('fuse_encoder.pkl', 'wb') as file:
    pickle.dump(encoder, file)


In [10]:
all_data = [(did[1],did[7],did[3].split(','),did[5],did[-1]) for did in data]
all_data[7]

('Benjamín Osorio Rico', 'Driving Mum', ['Drama', 'Comedy'], 'Estonia', 1)

In [11]:
all_ds = FuseDataset(all_data,encoder)

In [12]:
all_ds.data[7]

('Benjamín Osorio Rico', 'Driving Mum', ['Drama', 'Comedy'], 'Estonia', 1)

In [13]:
all_ds[7]

(tensor(8.), tensor(8.), tensor([1., 0., 0.,  ..., 0., 0., 0.]), tensor(1.))

In [19]:
print(len(all_ds[7][2]))

1770


In [26]:
train_ds = ConFDataset(train_data,encoder)
test_ds = ConFDataset(test_data,encoder)
validation_ds = ConFDataset(validation_data,encoder)

In [27]:
len(test_ds)

3701

In [28]:
BATCH_SIZE = 64
train_data_loader = DataLoader(train_ds,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)
validation_data_loader = DataLoader(validation_ds,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)
test_data_loader = DataLoader(test_ds,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)

In [29]:
train_ds[7]

(tensor(0),
 tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]),
 tensor(0.5000))

In [30]:
# for ds in test_data_loader:
#     print(ds)

In [49]:
print(len(train_ds[0][1]))
train_ds[0][1]

91


tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.])

In [48]:

num_users = len(encoder.vocab_to_idx['users'])
print(num_users)
#Encoding TODO stuff.
num_mv_features = len(train_ds[0][1])
print(num_mv_features)

FEATURES = 700


LEARNING_RATE = 0.001
#weight decay L2 regularization
# optimiser = optim.SGD(model.parameters(), lr=0.001,weight_decay=1e-5)\
L2_REGULARIZATION = 0.1
EPOCHS = 300

1100
91


In [33]:
import mlflow
from mlflow.tracking import MlflowClient

# Optional: Set MLflow experiment
mlflow.set_experiment("Finetuned Content Filter")

# Log model parameters (example)
mlflow.start_run()
mlflow.log_param("epochs", EPOCHS)
mlflow.log_param("optimiser", 'SGD')
mlflow.log_param("learning rate", LEARNING_RATE)
mlflow.log_param("batch size", BATCH_SIZE)
mlflow.log_param("L2 regularization", L2_REGULARIZATION)
mlflow.log_param("drop out", 0.5)

0.5

# KFOLDS CROSS VALIDATION

# PRETEST

In [34]:
# #test data
# model.eval()  # Switch to evaluation mode
# test_loss = 0
# correct_predictions = 0  # Fixed variable name for clarity
# total_contexts = 0

# with torch.no_grad():  # Disable gradient computation
#     for user_ids, movie_features, ratings in test_data_loader:
        
#         predictions = model(user_ids,movie_features)  # Generate predictions
#         # Calculate and accumulate loss
#         loss = criterion(predictions, ratings)
#         test_loss += loss.item()
        
#         # # Reshape predictions to match [batch_size, context_size, vocab_size]
#         # predictions = predictions.view(-1, context_size, VOCAB_SIZE)
        
#         # # Get top prediction for each context position
#         # top_predictions = predictions.argmax(dim=2)

#         # # Calculate correct predictions
#         # correct_preds = (top_predictions == context).float().sum()
#         # correct_predictions += correct_preds.item()  # Accumulate correct predictions
        
#         # total_contexts += context.numel()  # Total number of context word positions evaluated

# # Calculate final metrics
# test_loss /= len(test_data_loader)  # Average loss per batch
# # print('correct predictions = ',correct_predictions)
# # print('out of  = ',total_contexts)
# # accuracy = correct_predictions / total_contexts  # Compute accuracy

# # print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')

# # Evaluate the model
# # (This would involve using a separate validation set or performing cross-validation)
# print(test_loss)
# mlflow.log_metric('pretraining test loss',test_loss)

## wandb

In [35]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    entity="jcrich",
    project="content-filter",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LEARNING_RATE,
    "architecture": "content filter",
    "dataset": "imdb",
    "epochs": EPOCHS,
    }
)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jcrich. Use `wandb login --relogin` to force relogin


In [36]:
#SPLit
from sklearn.model_selection import KFold

# Configuration
num_folds = 10
data_size = len(test_ds)  # Assuming 'dataset' is a PyTorch Dataset

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

In [37]:
# Correct variable names and logic
# Set the best validation loss to infinity at the start

best_val_loss = float('inf')

for fold, (train_ids, val_ids) in enumerate(kfold.split(train_ds)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)

    trainloader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=train_subsampler)
    valloader = torch.utils.data.DataLoader(validation_ds, batch_size=BATCH_SIZE, sampler=val_subsampler)

    model = MLPContentFilter(num_users, num_mv_features, embedding_dim=FEATURES)
    optimiser = optim.SGD(model.parameters(), lr=LEARNING_RATE,weight_decay=L2_REGULARIZATION)
    criterion = nn.MSELoss()
    
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0.0
        #user,features, target
        for batch_idx, (user_ids, movie_features, ratings) in tqdm(enumerate(train_data_loader), total=len(train_data_loader), desc=f'Epoch {epoch}'):

            optimiser.zero_grad()
            outputs = model(user_ids, movie_features).squeeze()
            loss = criterion(outputs, ratings)
            loss.backward()
            optimiser.step()
            train_loss += loss.item()

        # Average training loss over all batches
        train_loss /= len(trainloader)
        # Log training loss for the current epoch
        wandb.log({"Avg. Training Loss": train_loss, "Epoch": epoch})
        mlflow.log_metric("Avg. Training Loss", train_loss, step=epoch)
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for user_indices, item_indices, ratings in valloader:
                outputs = model(user_indices, item_indices).squeeze()
                loss = criterion(outputs, ratings)
                val_loss += loss.item()

        avg_validation_loss = val_loss / len(valloader)
        print(f'Validation Loss for fold {fold}: {avg_validation_loss}')
        
        if avg_validation_loss < best_val_loss:
            best_val_loss = avg_validation_loss
            mlflow.log_metric("Best Validation Loss", best_val_loss, step=epoch)
            mlflow.pytorch.log_model(model, "model")
            # Save model state
            torch.save(model.state_dict(), 'best_model_state.pth')
            # If you also want to save the optimizer state along with the model:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimiser_state_dict': optimiser.state_dict(),
                'loss': avg_validation_loss,
            }, 'best_col_model_checkpoint.pth')
            # Log the model checkpoint as an artifact
            mlflow.log_artifact('best_col_model_checkpoint.pth')
        
        # Log validation loss for the current epoch
        wandb.log({"Avg Validation Loss": avg_validation_loss, "Epoch": epoch})
        mlflow.log_metric("Avg Validation Loss", avg_validation_loss, step=epoch)
    
    print('--------------------------------')

wandb.finish()

FOLD 0
--------------------------------


Epoch 0: 100%|█████████████████████████████████| 23/23 [00:00<00:00, 252.68it/s]


Validation Loss for fold 0: 0.3913260300954183


Epoch 1: 100%|█████████████████████████████████| 23/23 [00:00<00:00, 238.67it/s]


Validation Loss for fold 0: 0.3930666248003642


...

Validation Loss for fold 9: 0.06493554140130679


Epoch 299: 100%|███████████████████████████████| 23/23 [00:00<00:00, 298.07it/s]


Validation Loss for fold 9: 0.06579883893330891
--------------------------------


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Avg Validation Loss,▄▁▁▁▂▁▁▁▇▁▁▁▂▁▁▁█▂▁▁▁▁▁▁▃▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
Avg. Training Loss,▂▂▁▁▃▁▁▁█▁▁▁▂▂▁▁▂▁▁▁▂▁▁▁▂▂▁▁▂▁▁▁▂▂▁▁▂▁▁▁
Epoch,▂▃▅▇▁▃▅▇▁▄▅▇▂▃▆▇▁▄▅▇▂▃▅█▁▃▆▇▂▄▅█▂▃▆▇▁▄▅█
Avg Validation Loss,0.0658
Avg. Training Loss,0.08057
Epoch,299


In [38]:
'STUFF'

'STUFF'

In [39]:
#test data
model.eval()  # Switch to evaluation mode
test_loss = 0
correct_predictions = 0  # Fixed variable name for clarity
total_contexts = 0

with torch.no_grad():  # Disable gradient computation
    for user_ids, movie_features, ratings in test_data_loader:
        predictions = model(user_ids,movie_features)  # Generate predictions
        # Calculate and accumulate loss
        loss = criterion(predictions, ratings)
        test_loss += loss.item()
        
        # # Reshape predictions to match [batch_size, context_size, vocab_size]
        # predictions = predictions.view(-1, context_size, VOCAB_SIZE)
        
        # # Get top prediction for each context position
        # top_predictions = predictions.argmax(dim=2)
        
        # # Calculate correct predictions
        # correct_preds = (top_predictions == context).float().sum()
        # correct_predictions += correct_preds.item()  # Accumulate correct predictions
        
        # total_contexts += context.numel()  # Total number of context word positions evaluated

# Calculate final metrics
test_loss /= len(test_data_loader)  # Average loss per batch
# print('correct predictions = ',correct_predictions)
# print('out of  = ',total_contexts)
# accuracy = correct_predictions / total_contexts  # Compute accuracy

# print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')

# Evaluate the model
# (This would involve using a separate validation set or performing cross-validation)
print(test_loss)
mlflow.log_metric('evaluation posttraining test loss',test_loss)
mlflow.end_run()

0.06992023327110107


# Batch all the training data for final model.

In [40]:
len(data)

18504

In [41]:
all_ds = ConFDataset(data,encoder)
all_data_loader = DataLoader(train_ds,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)

In [42]:
final_model = MLPContentFilter(num_users, num_mv_features, embedding_dim=FEATURES)
optimiser = optim.SGD(final_model.parameters(), lr=LEARNING_RATE,weight_decay=L2_REGULARIZATION)

In [43]:
# Optional: Set MLflow experiment
mlflow.set_experiment("Finetuned Collaborative Filter")
# Log model parameters (example)
mlflow.start_run()
mlflow.log_param("epochs", EPOCHS)
mlflow.log_param("optimizer", type(optimiser).__name__)
mlflow.log_param("learning rate", LEARNING_RATE)
mlflow.log_param("batch size", BATCH_SIZE)
mlflow.log_param("L2 regularization", L2_REGULARIZATION)
mlflow.log_param("KFOLDS", num_folds)
mlflow.log_param("drop out", 0.5)

0.5

In [44]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    entity="jcrich",
    project="collaborative filter model",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": LEARNING_RATE,
    "architecture": "collaborative filter",
    "dataset": "letterboxd",
    "epochs": EPOCHS,
    }
)

In [45]:
# Initialize lists to keep track of losses and epochs
# Initialize MLflow run
# Initialize MLflow run
# Initialize MLflow run

# Ensure this is the model you intend to train
model = final_model

for epoch in range(EPOCHS):
    model.train()  # Correct model reference for training mode
    total_epoch_loss = 0
    for batch_idx, (user_ids, movie_features, ratings) in tqdm(enumerate(all_data_loader), total=len(all_data_loader), desc=f'Epoch {epoch}'):

        optimiser.zero_grad()
        predictions = model(user_ids, movie_features)  # Ensure consistent model reference
        loss = criterion(predictions, ratings)

        loss.backward()
        optimiser.step()
        
        batch_loss = loss.item()
        total_epoch_loss += batch_loss
        
    avg_epoch_loss = total_epoch_loss / len(all_data_loader)
    
    # Logging
    wandb.log({"Avg. Training Loss": avg_epoch_loss, "Epoch": epoch})
    mlflow.log_metric("Avg. Training Loss", avg_epoch_loss, step=epoch)

# Finish the Weights & Biases run
wandb.finish()

Epoch 299: 100%|███████████████████████████████| 23/23 [00:00<00:00, 304.36it/s]
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Avg. Training Loss,█▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Avg. Training Loss,0.06881
Epoch,299


In [46]:
torch.save({
    'epoch': epoch,
    'model_state_dict': final_model.state_dict(),
    'optimiser_state_dict': optimiser.state_dict(),
    'Avg. training loss': avg_epoch_loss,
}, 'final_con_model_checkpoint.pth')
mlflow.log_artifact('final_con_model_checkpoint.pth')
mlflow.pytorch.log_model(final_model, "model")
# Save model state
torch.save(final_model.state_dict(), 'best_model_state.pth')

In [47]:
#test data
final_model.eval()  # Switch to evaluation mode
test_loss = 0
correct_predictions = 0  # Fixed variable name for clarity
total_contexts = 0

with torch.no_grad():  # Disable gradient computation
    for user_ids, movie_ids, ratings in test_data_loader:
        predictions = final_model(user_ids,movie_ids)  # Generate predictions
        # Calculate and accumulate loss
        loss = criterion(predictions, ratings)
        test_loss += loss.item()
        
        # # Reshape predictions to match [batch_size, context_size, vocab_size]
        # predictions = predictions.view(-1, context_size, VOCAB_SIZE)
        
        # # Get top prediction for each context position
        # top_predictions = predictions.argmax(dim=2)
        
        # # Calculate correct predictions
        # correct_preds = (top_predictions == context).float().sum()
        # correct_predictions += correct_preds.item()  # Accumulate correct predictions
        
        # total_contexts += context.numel()  # Total number of context word positions evaluated

# Calculate final metrics
test_loss /= len(test_data_loader)  # Average loss per batch
# print('correct predictions = ',correct_predictions)
# print('out of  = ',total_contexts)
# accuracy = correct_predictions / total_contexts  # Compute accuracy

# print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}')

# Evaluate the model
# (This would involve using a separate validation set or performing cross-validation)
print(test_loss)

mlflow.log_metric('Post training test loss',test_loss)
mlflow.end_run()

0.07054703911406952


## Preference ranking

In [37]:
target_user_id = 140440102666832

In [38]:
e_uid = encoder.encode(target_user_id,encoder.user_to_idx)

AttributeError: 'Encoder' object has no attribute 'user_to_idx'

In [ ]:
def get_user_features(uid):

    for i in range(0,len(all_ds)):

        u,m,r = all_ds[i]
        # print(u)
        if u == uid:
            yield [m,r]
    

In [ ]:
def predict_ratings(e_uid,e_mid,model):
    e_uid_tensor = torch.tensor(e_uid, dtype=torch.int64).unsqueeze(0)
    movie_eid_tensor = torch.tensor(e_mid,dtype=torch.int64).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        user_rating = model(e_uid_tensor,movie_eid_tensor)
        return movie_eid_tensor,user_rating
        
    

In [ ]:
#get user and features
seen_features = set(tuple(fe) for fe in get_user_features(e_uid)) # Get user features if needed
seen_movies = set(sf[0].item() for sf in seen_features)
all_movies = set([num for num in range(30)])
unseen_movies = all_movies - seen_movies
unseen_features = {tuple(predict_ratings(e_uid,um,model)) for um in unseen_movies}
all_features = seen_features | unseen_features

sorted_set = sorted(all_features, key=lambda x: x[1])
top_n = 10
recommendations = list(filter(lambda x: x[0].item() in unseen_movies,sorted_set))
n_recommendations = list(map(lambda x: x[0],recommendations[-1:top_n*-1:-1]))
n_recommendations


# Matching

In [ ]:
#all users, with all movie ratings.
def generate_all_ratings(model,num_users,num_movies):
    #build matrix

    #data already encoded....
    all_data = [['dummy',mnm,unm,'dummy','0/10'] for unm in range(0,num_users) for mnm in range(0,num_movies)]
    # print(all_data)
    # print([em for em in all_data])
    fake_encoder = Encoder([did[2] for did in all_data],[did[1] for did in all_data])
    all_ds = CFDataset(all_data,fake_encoder)
    
    data_loader = DataLoader(all_ds, batch_size=1)
    for uid,mid,ra in data_loader:
        # print(uid,mid)
        prediction = model(uid,mid)
        yield (uid.item(),mid.item(),prediction.item())



In [ ]:
def create_ratings_tensor(data, num_users, num_movies):
    # Initialize an empty tensor to hold the ratings
    ratings_tensor = torch.zeros(num_users, num_movies)
    
    # Iterate over the data and fill the tensor
    for entry in data:
        user_id, movie_id, rating = entry
        # Convert rating to float
        rating = float(rating)
        ratings_tensor[user_id, movie_id] = rating
    
    return ratings_tensor

In [ ]:
# Step 1: Generate Predictions
predictions = [ra for ra in generate_all_ratings(model,num_users,num_movies)]
# print(predictions)
ratings_tensor = create_ratings_tensor(predictions,num_users,num_movies)
print(ratings_tensor)
# predictions[0]

# ratings_tensor.shape

# ratings_tensor[0]

import torch.nn.functional as F

# Assuming predicted_ratings is your tensor of predicted ratings
# predicted_ratings.shape should be (num_users, num_movies)

# # Calculate cosine similarity
# user_similarities = F.cosine_similarity(ratings_tensor, ratings_tensor, dim=1)

# user_similarities

## COSINE SIMILARITY

In [ ]:
# Step 1: Normalize predicted ratings
normalized_ratings = F.normalize(ratings_tensor, p=2, dim=1)

# Step 2: Calculate cosine similarity
user_similarities = torch.matmul(normalized_ratings, normalized_ratings.T)

# Set diagonal elements to a large negative value to exclude self-similarity
user_similarities.fill_diagonal_(-float('inf'))

# You can optionally convert the similarities tensor to a numpy array for easier manipulation
user_similarities_np = user_similarities.numpy()

# Print or use the user similarities tensor
print(user_similarities)

### Matches

In [ ]:
top_matches = {}
for i in range(len(user_similarities)):
    # Sort similarities for the current user i
    ranked_users = torch.argsort(torch.tensor(user_similarities[i]), descending=True)
    # Exclude self from top matches (optional)
    top_matches[i] = ranked_users[ranked_users != i]


In [ ]:
top_matches

In [ ]:
encoder.decode(71,encoder.idx_to_user)

In [ ]:
#decoded

decoded_matches = {encoder.decode(k,encoder.idx_to_user):[encoder.decode(ve,encoder.idx_to_user) for ve in v.tolist()] for k,v in top_matches.items()}
decoded_matches

In [ ]:
# Step 4: Select Top Matches
N = 5  # Number of top matches to select
for user, similar_users in decoded_matches.items():
    decoded_matches[user] = similar_users[:N]

decoded_matches

In [ ]:
#decode....

## FINALLY FIND THE MATCHES....

In [ ]:
target_user = 140440102666832



In [ ]:
decoded_matches[target_user]

In [ ]:
neighbour = 140440115331424

## Now see how they relate to the database...

In [ ]:
reviews = db.get_table_values('Reviews')

In [ ]:
#TODO

In [ ]:
def reviews_info(user_id,reviews):
    # print(db.get_table_values('Users'))
    info = {}
    info[user_id] = list(filter(lambda x :x[0] == user_id,db.get_table_values('Users')))[0][1]

    
    #TODO finish work...
    mov_tab = db.get_table_values('Movies')
    
    for review in reviews:
        mov_id = review[1]
        info[mov_id] = {'title':list(filter(lambda x :x[0] == mov_id,mov_tab))[0][1],'rating':review[4]}
        
    return info
    
    

In [ ]:
target_reviews = list(filter(lambda rv: rv[2] == target_user,reviews))
neighbour_reviews = list(filter(lambda rv: rv[2] == neighbour,reviews))

In [ ]:
tar_info = reviews_info(target_user,target_reviews)
neigh_info = reviews_info(neighbour,neighbour_reviews)

In [ ]:
tar_info

In [ ]:
neigh_info

# SAVE_MODEL

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')